# Q1

In [1]:
demographics = [80 34; 60 44; 40 44; 20 24; 40 114; 40 64; 70 14; 50 44; 70 54; 70 64]

10×2 Array{Int64,2}:
 80   34
 60   44
 40   44
 20   24
 40  114
 40   64
 70   14
 50   44
 70   54
 70   64

In [12]:
using JuMP, Cbc
m = Model(solver = CbcSolver())
@variable(m, district[1:10, 1:5], Bin)
@variable(m, dem_majority[1:5],Bin)

#only one district per city
for i in 1:10
    @constraint(m, sum(district[i,:]) == 1)
end

#district size requirement
for i in 1:5
    @constraint(m, sum(district[:,i] .* (demographics * [1; 1])) >= 150)
    @constraint(m, sum(district[:,i] .* (demographics * [1; 1])) <= 250)
end

#define dem majority
for i in 1:5
    # z(dem - rep - 1) >= 0
    @constraint(m, (dot(district[:,i] , 
                demographics[:,2]) - dot(district[:,i] , demographics[:,1]) - 1) >= (1 -dem_majority[i]) * -1000)
end

#maximize majorities
@objective(m, Max, sum(dem_majority))

solve(m)



:Optimal

In [14]:
println("Blue aquires the best of gerrymander by having democratic majority in ", getobjectivevalue(m) , 
    " districts, which is achieved by," )

district_vals = getvalue(district)
for i in 1:10
    for j in 1:5
        if district_vals[i,j] == 1
            println("Giving city " , i, " to district ", j)
        end
    end
end

Blue aquires the best of gerrymander by having democratic majority in 3.0 districts, which is achieved by,
Giving city 1 to district 5
Giving city 2 to district 5
Giving city 3 to district 2
Giving city 4 to district 2
Giving city 5 to district 1
Giving city 6 to district 4
Giving city 7 to district 3
Giving city 8 to district 2
Giving city 9 to district 4
